In [60]:
# LSTM weights, train_new_model csv
from google.colab import files

uploaded = files.upload()


Saving LSTM_sequence_weights.h5 to LSTM_sequence_weights.h5


In [3]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM, Lambda, Reshape
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

  
def model_LSTM(hidden_size):  
  model = Sequential()
  model.add(LSTM(hidden_size, return_sequences=False, input_shape=(Tx,1)))
  model.add(Dense(units=1, activation='sigmoid'))
  
  return model


hidden_size = 16

reshapor = Reshape((1, 1)) 
LSTM_cell = LSTM(hidden_size, return_state = True) 
densor = Dense(units=1, activation='sigmoid') 

def test_model(Tx, hidde_size):
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, 1))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(hidden_size,), name='a0')
    c0 = Input(shape=(hidden_size,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
        
    # Step 3: Create model instance
    model = Model(inputs=[X,a0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model


Using TensorFlow backend.


In [0]:
Tx = 6
Ty = 1

In [0]:
model = test_model(Tx, hidden_size)
#model.summary()

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.load_weights('LSTM_sequence_weights.h5')

In [0]:
import pandas as pd
import numpy as np

def scale_substraction_mean_for_1sample(X):
  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
    mean = np.mean(line)
    #print(mean)
    X[i] = line - mean 
    #if min(X[i]) < 0:
    #  X[i] -= min(X[i])*1.2
    #print(X[i])
    
  return X
    

In [0]:
data_y0 = pd.read_csv('train_y0_v8.csv')
X_0 = data_y0.drop(['Y'], axis='columns').values
y_0 = data_y0['Y'].values
test_0 =  data_y0.drop(['Y'], axis='columns').values

X_0 = scale_substraction_mean_for_1sample(X_0)
X_0 = np.expand_dims(X_0, axis=2)

In [0]:
data_y1 = pd.read_csv('train_y1_v8.csv')
X_1 = data_y1.drop(['Y'], axis='columns').values
y_1 = data_y1['Y'].values
test_1 =  data_y1.drop(['Y'], axis='columns').values

X_1 = scale_substraction_mean_for_1sample(X_1)
X_1 = np.expand_dims(X_1, axis=2)

In [34]:
X = X_1
test = test_1
counter = 0
min_pred = []
for i in range(X.shape[0]):
  x_ini = np.zeros((1,6,1))
  #print(test[i].reshape(1,-1))
  
  x_ini[0,:,:] = X[i]
  a_ini = np.zeros((1, hidden_size))
  c_ini = np.zeros((1, hidden_size))
  pred = model.predict([x_ini,a_ini,c_ini])
  min_pred.append(np.min(pred[1:]))
  
  for k in range(len(pred)):
    if pred[-1] < 0.5:
      counter += 1
    #print(pred[k][0])
pos = np.argmin(min_pred)  
print(np.min(min_pred))
a_ini = np.zeros((1, hidden_size))
c_ini = np.zeros((1, hidden_size))
x_ini = np.zeros((1,6,1))
x_ini[0,:,:] = X[pos]
print(model.predict([x_ini,a_ini,c_ini]))

#

2.2023356e-07
[array([[0.06029459]], dtype=float32), array([[0.19785048]], dtype=float32), array([[0.00025734]], dtype=float32), array([[2.2023356e-07]], dtype=float32), array([[2.468872e-05]], dtype=float32), array([[0.98965454]], dtype=float32)]


In [35]:
print(counter)
print(pos)

0
9682


In [36]:
print(X[pos].tolist())
print(test_1[pos])
print(y_1[pos])


[[0.2578165000000323], [0.16087950000000517], [0.05855750000004556], [-0.04979750000003058], [-0.1597424999999577], [-0.26771350000009364]]
[2.205745 2.108808 2.006486 1.898131 1.788186 1.680215]
1.0


In [67]:
print(test_1.shape)

(9926, 6)


In [21]:
X = X_1
test = test_1
for i in range(300, 400):
  x_ini = np.zeros((1,6,1))
  x_ini[0,:,:] = X[i]
  a_ini = np.zeros((1, hidden_size))
  c_ini = np.zeros((1, hidden_size))
  pred = model.predict([x_ini,a_ini,c_ini])
  
  print(X[i].tolist())
  for el in pred:
    print(el)


[[0.026798499999998338], [0.0753885000000018], [0.12856850000001313], [0.1820005000000699], [0.24331349999989776], [0.3086764999999591]]
[[0.9582819]]
[[0.9880995]]
[[0.95545924]]
[[0.99673754]]
[[0.996062]]
[[0.86421496]]
[[0.02881013333333468], [0.08199013333334601], [0.13542213333340322], [0.19673513333323064], [0.26209813333329246], [0.33210913333344216]]
[[0.9586835]]
[[0.9889324]]
[[0.9504947]]
[[0.99856514]]
[[0.9957824]]
[[0.9285435]]
[[0.030835199999996815], [0.08426720000005403], [0.14558019999988145], [0.21094319999994235], [0.2809542000000925], [0.35748719999991885]]
[[0.95904607]]
[[0.9888658]]
[[0.9561713]]
[[0.9975781]]
[[0.9956434]]
[[0.92839307]]
[[0.03369953333332143], [0.09501253333314885], [0.1603755333332102], [0.2303865333333599], [0.30691953333318667], [0.3867895333333422]]
[[0.95948976]]
[[0.99003077]]
[[0.9509632]]
[[0.9983736]]
[[0.99429095]]
[[0.9396757]]
[[0.03615693333335468], [0.10151993333341647], [0.17153093333356617], [0.24806393333339294], [0.327933933